# 양방향 RNN을 이용한 품사 태깅

PyTorch 시퀸스 레이블링의 대표적 테스크 품사 태깅 (PoS tagging)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data, datasets
import time
import random

In [8]:
# 랜덤 시드 고정
# 랜덤 시드 1234보다 3407이 정확도 면에서 훨씬 좋다
SEED = 3407
random.seed(SEED)
torch.manual_seed(SEED)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 필드 정의하기

총 3개의 필드 정의

In [11]:
TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbbags", PTB_TAGS))

# 2. 데이터셋 만들기

1. 훈련 데이터

2. 검증 데이터

3. 테스트 데이터

In [12]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


100%|██████████| 688k/688k [00:01<00:00, 562kB/s] 


extracting
